# Google Drive Torrent Downloader

This script allows downloading torrents directly to Google Drive through Google Colab.

⚠️ **Important**: For research and educational purposes only. Please respect copyright laws and use only for legal content.

## 1. Install Dependencies

In [ ]:
# Install dependencies (fixed version)
!apt-get update
!apt-get install -y libtorrent-rasterbar-dev
!pip install libtorrent

# Check installation
import libtorrent as lt
print(f"✅ LibTorrent version: {lt.version}")
print("✅ Installation completed successfully!")

## 2. Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create folder for torrents
import os
torrent_dir = '/content/drive/MyDrive/Torrents'
os.makedirs(torrent_dir, exist_ok=True)
print(f"Torrents folder: {torrent_dir}")

## 3. Torrent Downloader

In [ ]:
import libtorrent as lt
import time
import os
from IPython.display import clear_output

class TorrentDownloader:
    def __init__(self, download_path):
        self.download_path = download_path
        self.session = lt.session()
        self.session.listen_on(6881, 6891)
        
    def download_torrent(self, torrent_file_or_magnet):
        """Download torrent by file or magnet link"""
        try:
            # Check if it's a magnet link or file
            if torrent_file_or_magnet.startswith('magnet:'):
                handle = lt.add_magnet_uri(self.session, torrent_file_or_magnet, {
                    'save_path': self.download_path
                })
            else:
                # Load torrent file
                info = lt.torrent_info(torrent_file_or_magnet)
                handle = self.session.add_torrent({
                    'ti': info,
                    'save_path': self.download_path
                })
            
            print(f"Starting download: {handle.name()}")
            
            # Progress monitoring
            while not handle.is_seed():
                status = handle.status()
                
                clear_output(wait=True)
                print(f"Torrent: {handle.name()}")
                print(f"Progress: {status.progress * 100:.1f}%")
                print(f"Download rate: {status.download_rate / 1000:.1f} KB/s")
                print(f"Upload rate: {status.upload_rate / 1000:.1f} KB/s")
                print(f"Peers: {status.num_peers}")
                print(f"Seeds: {status.num_seeds}")
                
                if status.state == lt.torrent_status.downloading:
                    print("Status: Downloading")
                elif status.state == lt.torrent_status.finished:
                    print("Status: Finished")
                    break
                elif status.state == lt.torrent_status.seeding:
                    print("Status: Seeding")
                    break
                
                time.sleep(2)
            
            print(f"\n✅ Download completed: {handle.name()}")
            print(f"Files saved to: {self.download_path}")
            
        except Exception as e:
            print(f"❌ Download error: {e}")
    
    def list_files(self):
        """Show downloaded files"""
        print("\n📁 Downloaded files:")
        for root, dirs, files in os.walk(self.download_path):
            level = root.replace(self.download_path, '').count(os.sep)
            indent = ' ' * 2 * level
            print(f"{indent}{os.path.basename(root)}/")
            sub_indent = ' ' * 2 * (level + 1)
            for file in files:
                file_path = os.path.join(root, file)
                size = os.path.getsize(file_path)
                size_mb = size / (1024 * 1024)
                print(f"{sub_indent}{file} ({size_mb:.1f} MB)")

# Create downloader instance
downloader = TorrentDownloader(torrent_dir)
print("✅ Torrent downloader ready!")

## 4. Usage

In [ ]:
# Option 1: Download by magnet link
magnet_link = "magnet:?xt=urn:btih:HASH&dn=filename"  # Replace with your magnet link
# downloader.download_torrent(magnet_link)

# Option 2: Download by torrent file
# First upload torrent file to Colab
from google.colab import files
print("Select torrent file for download:")
uploaded = files.upload()

# Get uploaded file name
torrent_file = list(uploaded.keys())[0]
print(f"Uploaded file: {torrent_file}")

# Start download
downloader.download_torrent(torrent_file)

## 5. View Downloaded Files

In [ ]:
# Show all downloaded files
downloader.list_files()

## 6. Manual Download (Alternative Method)

In [ ]:
# For direct usage without class
def download_torrent_simple(magnet_or_file, save_path):
    import libtorrent as lt
    
    session = lt.session()
    session.listen_on(6881, 6891)
    
    if magnet_or_file.startswith('magnet:'):
        handle = lt.add_magnet_uri(session, magnet_or_file, {'save_path': save_path})
    else:
        info = lt.torrent_info(magnet_or_file)
        handle = session.add_torrent({'ti': info, 'save_path': save_path})
    
    print(f"Downloading: {handle.name()}")
    
    while not handle.is_seed():
        status = handle.status()
        print(f"\rProgress: {status.progress * 100:.1f}%", end='', flush=True)
        
        if status.state == lt.torrent_status.finished:
            break
            
        time.sleep(1)
    
    print(f"\n✅ Download completed!")

# Usage example:
# download_torrent_simple("magnet:?xt=urn:btih:HASH", "/content/drive/MyDrive/Torrents")

## 📝 Notes

- Files will be saved to `/content/drive/MyDrive/Torrents/` folder on your Google Drive
- Colab session may disconnect after 12 hours of inactivity
- For large files, it's recommended to check progress regularly
- **For research and educational purposes only**
- Use only for legal content and respect copyright laws
- If you encounter connection issues, try restarting the dependencies installation cell